### The script in the cell below will create an animated 3d graph of soil water content in pylab.  

However, Jupyter has a static connection with the code and the dispaly so it won't animate in here.

To get it to animate include the line:
%%writefile C:\Anaconda\Lib\SoilAnimation.py
at the top of this cell.  This will create a file called SoilAnimation.py
then go to the ipython interface and type:
import SoilAnimation.py

and it will display the animated graph in a Pylab window

In [117]:
#%%writefile C:\Anaconda\Lib\SoilAnimation.py
import sqlite3
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

con = sqlite3.connect('C:\ApsimX\Prototypes\MultiPoreWater/ReportDetail.db')
APSIMData = pd.read_sql("Select * from Report,Simulations where Report.SimulationID = Simulations.ID",con, index_col = ['TimeStep','Date','Process','Hour','Layer'])
APSIMData = APSIMData.drop(labels = ['SimulationID', 'ZoneName','ID'],axis=1)
PoreWater = APSIMData.ix[:,[X for X in APSIMData.columns if 'PW' in X]]
LayerNum = 7
PoreComNum = 7

LayerIndices = []
for i in range(1,LayerNum +1):
    LayerIndices.append('L'+str(i))
    
CompartmentIndices = []
for i in range(1,PoreComNum +1):
    CompartmentIndices.append('C'+str(i))    

PoreColumnIndex = pd.MultiIndex.from_product([LayerIndices,CompartmentIndices],names=['Layer','PoreComponent'])
PoreWater.columns = PoreColumnIndex

def WetnessData(i):
    Wetness = []
    for l in range(1,LayerNum+1):
        Wetness.append(list(PoreWater.ix[i+1,'L'+str(l)].values[0]))
    return np.array(Wetness)

fig = plt.figure()
ax = axes3d.Axes3D(fig)

xs = range(1,LayerNum+1)
ys = range(1,PoreComNum+1)
X, Y = np.meshgrid(xs, ys)
Z = WetnessData(1)
wframe = ax.plot_wireframe(X, Y, Z, rstride=2, cstride=2)
ax.set_zlim(-1,1)

def update(i, ax, fig):
    ax.cla()
    Z = WetnessData(i)
    wframe = ax.plot_wireframe(X, Y, Z, rstride=2, cstride=2)
    ax.set_zlim(-1,1)
    return wframe,

ani = animation.FuncAnimation(fig, update, 
        frames=xrange(1000), 
        fargs=(ax, fig), interval=10)
plt.show()

Writing C:\Anaconda\Lib\SoilAnimation.py
